# Importing Libraries

In [2]:
import pickle
import numpy as np
import pandas as pd
from PIL import Image
import albumentations as A
from IPython.display import SVG
import matplotlib.pyplot as plt
%matplotlib inline
import os, re, sys, random, shutil, cv2

import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras.models import Model
from keras.optimizers import Adam, Nadam
from keras import applications, optimizers
from keras.applications import InceptionResNetV2
from keras.applications import resnet

from keras.preprocessing.image import ImageDataGenerator
from keras.utils import model_to_dot, plot_model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger, LearningRateScheduler
from keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, ZeroPadding2D, Dropout

# Data Augmentation

In [3]:
images_dir = './data/images/'
masks_dir = './data/masks/'

In [8]:
file_names = np.sort(os.listdir(images_dir))
file_names = np.char.split(file_names, '.')
filenames = np.array([])
for i in range(len(file_names)):
    filenames = np.append(filenames, file_names[i][0])

In [10]:
transform_1 = A.Compose([
    A.HorizontalFlip(p=1.0),
    A.VerticalFlip(p=1.0),
    A.Rotate(limit=[60, 300], p=1.0, interpolation=cv2.INTER_NEAREST),
    A.OneOf([
        A.GridDistortion(distort_limit= 0.2,p=0.5),
        A.OpticalDistortion(distort_limit=1, shift_limit=0.5, interpolation=cv2.INTER_NEAREST, p=0.5),
    ], p=1.0),
], p=1.0)

transform_2 = A.Compose([
   A.RandomBrightnessContrast(brightness_limit=[-0.05, 0.20], contrast_limit=0.2, p=1.0),
    A.OneOf([
        A.CLAHE (clip_limit=1.5, tile_grid_size=(8, 8), p=0.5),
    ], p=1.0),
], p=1.0)

In [13]:
for i in range(8):
    print(f"loop {i+1} ...")
    for file in filenames:
        img = cv2.imread(images_dir+file+'.png')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(masks_dir+'mask'+file[5:]+'.png')
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)

        transformed = transform_1(image=img, mask=mask)
        transformed_image = transformed['image']
        transformed_mask = transformed['mask']

        cv2.imwrite('./data/images/aug_image{}_'.format(str(i+1))+file[5:]+'.png', cv2.cvtColor(transformed_image, cv2.COLOR_BGR2RGB))
        cv2.imwrite('./data/masks/aug_mask{}_'.format(str(i+1))+file[5:]+'.png', cv2.cvtColor(transformed_mask, cv2.COLOR_BGR2RGB))

for i in range(8, 10):
    print(f"loop {i+1} ...")
    for file in filenames:
        img = cv2.imread(images_dir+file+'.png')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(masks_dir+'mask'+file[5:]+'.png')
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)

        transformed = transform_2(image=img)
        transformed_image = transformed['image']
        transformed_mask = mask

        cv2.imwrite('./data/images/aug_image{}_'.format(str(i+1))+file[5:]+'.png', cv2.cvtColor(transformed_image, cv2.COLOR_BGR2RGB))
        cv2.imwrite('./data/masks/aug_mask{}_'.format(str(i+1))+file[5:]+'.png', cv2.cvtColor(transformed_mask, cv2.COLOR_BGR2RGB))

loop 1 ...
loop 2 ...
loop 3 ...
loop 4 ...
loop 5 ...
loop 6 ...
loop 7 ...
loop 8 ...
loop 9 ...
loop 10 ...


In [86]:
mask_segment = np.zeros(mask.shape, dtype=np.uint8)
mask_segment[mask < 32] = 0
mask_segment[mask >= 32] = 1
mask_segment[mask >= 95] = 2
mask_segment[mask >= 159] = 3
mask_segment[mask >=223] = 4
mask_segment = mask_segment[:,:,0]